In [1]:
import os 
import pandas as pd
import numpy as np
import glob
import pyodbc
import gc

# Housholds

In [3]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                    'Server=DDAMWSQL16.sandag.org;'
                    'Database=estimates;'
                    'Trusted_Connection=yes;')

with open(r'C:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2023\2023-024 Estimates 2022\series_14_automation\sql_queries\household_type_query_2021.sql', 'r') as sql_file:
    sql_query = sql_file.read()

df =  pd.read_sql_query(sql_query, conn)
df

,mgra_id,yr_id,household_size_id,households
0,1401703001,2020,1,6
1,1401703001,2020,2,12
2,1401703001,2020,3,5
3,1401703001,2020,4,1
4,1401703001,2020,5,2
...,...,...,...,...
382447,1401874702,2021,3,0
382448,1401874702,2021,4,0
382449,1401874702,2021,5,0
382450,1401874702,2021,6,0


In [4]:
geo_level = 'mgra_id'
output = df.groupby([f'{geo_level}', 'yr_id', 'household_size_id']).sum().reset_index()
output = pd.pivot(output, values='households', index=[f'{geo_level}', 'yr_id'], columns=['household_size_id'])
output['Total Households'] = output.sum(axis=1)
output.columns.name = ''
mgra_id_households_data = output.reset_index()

mgra_id_households_data

,mgra_id,yr_id,1,2,3,4,5,6,7,Total Households
0,1400000101,2020,5,5,4,0,1,2,1,18
1,1400000101,2021,5,5,4,0,2,1,1,18
2,1400000102,2020,0,0,0,0,0,0,0,0
3,1400000102,2021,0,0,0,0,0,0,0,0
4,1400000201,2020,13,11,5,4,0,0,1,34
...,...,...,...,...,...,...,...,...,...,...
54631,1402300001,2021,31,47,24,13,6,3,2,126
54632,1402300101,2020,23,29,12,8,4,3,1,80
54633,1402300101,2021,25,30,12,5,3,3,2,80
54634,1402300201,2020,0,0,0,0,0,0,0,0


In [5]:
mgra_id_households_data.to_csv(rf'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-023 Estimates 2022\Results\Test 5- Vintage Comparison\2021_01\households\QA_2021_01_mgra_id_households.csv', index=False)

In [10]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                    'Server=DDAMWSQL16.sandag.org;'
                    'Database=estimates;'
                    'Trusted_Connection=yes;')

with open(r'C:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2023\2023-024 Estimates 2022\series_14_automation\sql_queries\mgra_denormalize_2021.sql', 'r') as sql_file:
    sql_query = sql_file.read()

mgra_denorm_2021 =  pd.read_sql_query(sql_query, conn)
mgra_denorm_2021['region'] = 'San Diego'
mgra_denorm_2021

,mgra_id,mgra,series,region,sra,tract,supervisorial,college,cpa,jurisdiction,zip,secondary,elementary,msa,sdcouncil,transit,unified,jurisdiction_id,cpa_id,taz
0,1000001,1,10,San Diego,Central San Diego,1,4,San Diego,Uptown,San Diego,92103,None,None,Central,2,None,San Diego,14,1442.0,NaN
1,1000002,2,10,San Diego,Central San Diego,1,4,San Diego,Uptown,San Diego,92103,None,None,Central,2,None,San Diego,14,1442.0,NaN
2,1000003,3,10,San Diego,Central San Diego,1,4,San Diego,Uptown,San Diego,92103,None,None,Central,2,None,San Diego,14,1442.0,NaN
3,1000004,4,10,San Diego,Central San Diego,1,4,San Diego,Uptown,San Diego,92103,None,None,Central,2,None,San Diego,14,1442.0,NaN
4,1000005,5,10,San Diego,Central San Diego,1,4,San Diego,Uptown,San Diego,92103,None,None,Central,2,None,San Diego,14,1442.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129893,1402299801,22998,14,San Diego,Carlsbad,221,5,Mira Costa Community College,*Not in a CPA*,Carlsbad,92011,None,None,North County West,None,NCTD,Carlsbad,1,0.0,1290.0
129894,1402299901,22999,14,San Diego,Carlsbad,221,5,Mira Costa Community College,*Not in a CPA*,Carlsbad,92011,None,None,North County West,None,NCTD,Carlsbad,1,0.0,1290.0
129895,1402300001,23000,14,San Diego,Carlsbad,221,5,Mira Costa Community College,*Not in a CPA*,Carlsbad,92011,None,None,North County West,None,NCTD,Carlsbad,1,0.0,1290.0
129896,1402300101,23001,14,San Diego,Carlsbad,221,5,Mira Costa Community College,*Not in a CPA*,Carlsbad,92011,None,None,North County West,None,NCTD,Carlsbad,1,0.0,1254.0


In [8]:
def household_type_2021_01_cleaning(df, geo_level):
    age_output = df.merge(mgra_denorm_2021[['mgra_id', geo_level]], how='left', on='mgra_id')
    age_output = age_output.drop('mgra_id', axis=1)
    age_output = age_output.groupby([geo_level, 'yr_id']).sum()
    #age_output['Total_Household_Type'] = age_output.sum(axis=1)
    # first_column = age_output.pop('Under 5')
    # age_output.insert(0, 'Under 5', first_column)

    return age_output

In [14]:
geo_levels = ['cpa', 'jurisdiction', 'mgra', 'region', 'sra']

In [16]:
for geo_level in geo_levels:
    household_type_2021_01_cleaning(df=mgra_id_households_data, geo_level=geo_level).to_csv(rf"C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-023 Estimates 2022\Results\Test 5- Vintage Comparison\2021_01\households\QA_2021_01_{geo_level}_households.csv")
    print(f"{geo_level} is done")

cpa is done
jurisdiction is done
mgra is done
region is done
sra is done
